In [ ]:
import tensorflow as tf
import os
import zipfile
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
import random
import shutil
import seaborn as sns
from PIL import Image

In [ ]:
cd /home/

In [ ]:
filezip = '/content/drive/My Drive/Salinan shopee-product-detection-dataset.zip'
buka_zip = zipfile.ZipFile(filezip, 'r')
buka_zip.extractall('/home/')
buka_zip.close()

In [ ]:
item = 'train/train'

In [ ]:
a = []
for i in sorted(os.listdir('/home/train/train')):
    a.append(len(os.listdir(f'/home/train/train/{i}')))

In [ ]:
a = np.array(a)

In [ ]:
l = np.array(sorted(os.listdir('/home/train/train')))[a<np.mean(a)]

In [ ]:
c = a[a<np.mean(a)]

In [ ]:
for i in range(len(c)):
    os.chdir(f'/home/train/train/{l[i]}')
    d = os.listdir(f'/home/train/train/{l[i]}')
    if i != 5:
        for j in range(int(((np.mean(a))-c[i])/2)):
            im=Image.open(d[j])
            im=im.convert("RGB")
            im_blur=im.filter(ImageFilter.GaussianBlur)
            im_unsharp=im.filter(ImageFilter.UnsharpMask)
            im_blur.save(str(j)+'bl_'+d[j])
            im_unsharp.save(str(j)+'un_'+d[j])

In [ ]:
cd /home/train/train/33

In [ ]:
n = os.listdir()
for i in range(2):
    for j in range(len(n)):
        im=Image.open(n[j])
        im=im.convert("RGB")
        im_blur=im.filter(ImageFilter.GaussianBlur)
        im_unsharp=im.filter(ImageFilter.UnsharpMask)
        im_blur.save(str(i)+'bl_'+n[j])
        im_unsharp.save(str(i)+'un_'+n[j])

In [ ]:
cd /home/

In [ ]:
datagen = image.ImageDataGenerator(
                    rescale=1./255,
                    validation_split=0.2,
                    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

In [ ]:
train_generator = datagen.flow_from_directory(
        item,  
        target_size=(224, 224),  
        batch_size=32,
        class_mode='categorical',
        subset='training', 
        shuffle=True)

test_generator = datagen.flow_from_directory(
        item, 
        target_size=(224, 224), 
        batch_size=32, 
        class_mode='categorical',
        subset='validation',
        shuffle=True)

In [ ]:
model = tf.keras.applications.MobileNetV2(weights=None, classes=42, pooling='max')

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.SGD(),
              metrics=['accuracy'])

In [ ]:
model.load_weights('/model(1).h5')

In [ ]:
model.summary()

In [ ]:
%%time
model.fit(
      train_generator,
      workers=4,
      steps_per_epoch=50,
      epochs=200,
      validation_data=test_generator, 
      validation_steps=10,
      verbose=1)